In [ ]:
%pip install numpy

In [1]:
import numpy as np
import random
import math

# Define the number of users and items
num_users = 11
num_items = 15
min_items_rated = 5

# Generate random user ratings
user_ratings = {}
# for user_id in range(1, num_users + 1):
#     user_ratings[f'User{user_id}'] = {f'Item{i}': random.randint(1, 5) for i in range(1, random.randint(1, num_items + 1))}
for user_id in range(1, num_users + 1):
    num_rated_items = random.randint(min_items_rated, num_items)
    rated_items = random.sample(range(1, num_items + 1), num_rated_items)
    user_ratings[f'User{user_id}'] = {f'Item{i}': random.randint(1, 5) for i in rated_items}


print(user_ratings)
# Get unique items sorted
items = sorted(list(set([item for user in user_ratings for item in user_ratings[user]])), key=lambda x: int(x[4:]))
print(items)

# Get unique users
users = sorted(list(user_ratings.keys()), key=lambda x: int(x[4:]))
print(users)

# Create matrix
matrix = np.zeros((num_users, num_items))

# Fill in the matrix with ratings
for i, user in enumerate(users):
    for j, item in enumerate(items):
        if item in user_ratings[user]:
            matrix[i, j] = user_ratings[user][item]

print(matrix)


{'User1': {'Item15': 4, 'Item7': 1, 'Item11': 1, 'Item10': 1, 'Item6': 3, 'Item3': 4, 'Item2': 3, 'Item1': 1}, 'User2': {'Item8': 1, 'Item4': 5, 'Item7': 3, 'Item11': 5, 'Item14': 2, 'Item6': 2, 'Item9': 5, 'Item5': 1, 'Item10': 4, 'Item1': 3, 'Item12': 3, 'Item2': 1, 'Item3': 5, 'Item15': 2}, 'User3': {'Item5': 5, 'Item14': 1, 'Item8': 2, 'Item11': 1, 'Item12': 4, 'Item1': 5, 'Item7': 4, 'Item9': 1, 'Item15': 2, 'Item10': 2, 'Item6': 1}, 'User4': {'Item8': 5, 'Item14': 3, 'Item11': 1, 'Item7': 4, 'Item9': 1, 'Item4': 2, 'Item2': 1}, 'User5': {'Item6': 2, 'Item3': 3, 'Item12': 3, 'Item2': 4, 'Item1': 3, 'Item13': 1, 'Item15': 5, 'Item10': 2, 'Item5': 5, 'Item14': 4}, 'User6': {'Item15': 2, 'Item11': 2, 'Item7': 3, 'Item3': 5, 'Item1': 3, 'Item5': 4, 'Item13': 3, 'Item14': 1, 'Item6': 3, 'Item8': 1, 'Item12': 1, 'Item4': 3, 'Item10': 5}, 'User7': {'Item6': 3, 'Item14': 1, 'Item5': 5, 'Item10': 1, 'Item7': 1, 'Item4': 4, 'Item2': 5, 'Item12': 4, 'Item13': 4, 'Item15': 2, 'Item8': 4, 'Ite

In [41]:
# Oriented towards the user
def find_k_nearest_neighbors(matrix, user_id, k):
    # Find the user's row
    user_index = users.index(user_id)
    user_row = matrix[user_index]

    # Calculate the similarities using MSD
    similarities = []

    for i, other_row in enumerate(matrix):
        if i == user_index:
            continue
        similarity = round(np.sum((user_row - other_row) ** 2)/len(user_row), 2)
        similarities.append((i, similarity))


    print("Similarities: ", similarities)

    # Sort the similarities and get the index of top k
    similarities.sort(key=lambda x: x[1])
    neighbors = similarities[:k]

    return neighbors

In [44]:
user_id = "User4" 
k = 3

In [45]:
neighbors = find_k_nearest_neighbors(matrix, user_id, k)
print("Neighbors: ", neighbors)

print(f"{user_id}:", matrix[users.index(user_id)])
for neighbor, _ in neighbors:
    print(f"Neighbor {neighbor}:", matrix[neighbor])


Similarities:  [(0, 9.27), (1, 10.47), (2, 7.67), (4, 8.4), (5, 5.73), (6, 10.07), (7, 9.0), (8, 4.47), (9, 7.6), (10, 10.27)]
Neighbors:  [(8, 4.47), (5, 5.73), (9, 7.6)]
User4: [2. 0. 0. 4. 0. 0. 5. 1. 0. 3. 0. 1. 1. 5. 3.]
Neighbor 8: [1. 1. 2. 5. 3. 1. 2. 3. 3. 2. 3. 3. 3. 2. 2.]
Neighbor 5: [2. 1. 0. 0. 1. 5. 4. 5. 4. 1. 0. 0. 3. 5. 2.]
Neighbor 9: [0. 4. 5. 3. 0. 3. 1. 2. 4. 2. 0. 2. 5. 3. 5.]


In [46]:
# find Aggregate rating of unrated items
def find_aggregate_ratings(matrix, user_id, k):
    # Find the user's row
    user_index = users.index(user_id)
    user_row = matrix[user_index]

    # Find the k nearest neighbors
    neighbors = find_k_nearest_neighbors(matrix, user_id, k)

    # Find the unrated items
    unrated_items = np.where(user_row == 0)[0]
    print("Unrated items: ", unrated_items)

    # Calculate the aggregate ratings
    aggregate_ratings = []

    for item in unrated_items:
        ratings = []
        for neighbor, _ in neighbors:
            if matrix[neighbor, item] != 0:
                ratings.append(matrix[neighbor, item])
        if ratings:
            aggregate_rating = round(np.mean(ratings), 2)
            aggregate_ratings.append((item, aggregate_rating))
    
    return aggregate_ratings

In [49]:
print(find_aggregate_ratings(matrix, user_id, 3))

Similarities:  [(0, 9.27), (1, 10.47), (2, 7.67), (4, 8.4), (5, 5.73), (6, 10.07), (7, 9.0), (8, 4.47), (9, 7.6), (10, 10.27)]
Unrated items:  [ 1  2  4  5  8 10]
[(1, 2.0), (2, 3.5), (4, 2.0), (5, 3.0), (8, 3.67), (10, 3.0)]


In [62]:
def find_aggregate_ratings_by_weighted_sum(matrix, user_id, k):
    # Find the user's row
    user_index = users.index(user_id)
    user_row = matrix[user_index]

    # Find the k nearest neighbors
    neighbors = find_k_nearest_neighbors(matrix, user_id, k)

    # Find the unrated items
    unrated_items = np.where(user_row == 0)[0]
    print("Unrated items: ", unrated_items)

    # Calculate the aggregate ratings
    aggregate_ratings = []

    for item in unrated_items:
        ratings = []
        weights = []
        for neightbor, similarity in neighbors:
            if matrix[neightbor, item] != 0:
                # print(neighbor, matrix[neightbor, item])
                ratings.append(matrix[neightbor, item])
                weights.append(similarity)
        if ratings:
            aggregate_rating = round(np.sum([rating * weight for rating, weight in zip(ratings, weights)])/sum(weights), 2)
            aggregate_ratings.append((item, aggregate_rating))
            
    return aggregate_ratings

In [63]:
print(find_aggregate_ratings_by_weighted_sum(matrix, user_id, 3))

Similarities:  [(0, 9.27), (1, 10.47), (2, 7.67), (4, 8.4), (5, 5.73), (6, 10.07), (7, 9.0), (8, 4.47), (9, 7.6), (10, 10.27)]
Unrated items:  [ 1  2  4  5  8 10]
[(1, 2.28), (2, 3.89), (4, 1.88), (5, 3.14), (8, 3.75), (10, 3.0)]


In [32]:
item_id = "Item1"
k = 3

In [33]:
# Oriented towards the item
def find_k_nearest_neighbors_item(matrix, item_id, k):
    # Find the item's column
    item_index = items.index(item_id)
    item_column = matrix[:, item_index]

    # Calculate the similarities using MSD
    similarities = []

    for i, other_column in enumerate(matrix.T):
        if i == item_index:
            continue
        similarity = round(np.sum((item_column - other_column) ** 2)/len(item_column), 2)
        similarities.append((i, similarity))

    print("Similarities: ", similarities)

    # Sort the similarities and get the index of top k
    similarities.sort(key=lambda x: x[1])
    neighbors = similarities[:k]

    return neighbors

In [35]:
neighbors = find_k_nearest_neighbors_item(matrix, item_id, k)
print("Neighbors: ", neighbors)

print(f"{item_id}:", matrix[:, items.index(item_id)])
for neighbor, _ in neighbors:
    print(f"Neighbor {neighbor}:", matrix[:, neighbor])

Similarities:  [(1, 4.0), (2, 4.0), (3, 2.27), (4, 5.73), (5, 6.64), (6, 5.73), (7, 3.82), (8, 4.18), (9, 7.55), (10, 2.27), (11, 5.27), (12, 8.18), (13, 2.45), (14, 4.91)]
Neighbors:  [(3, 2.27), (10, 2.27), (13, 2.45)]
Item1: [2. 1. 0. 5. 0. 0. 3. 3. 3. 2. 4.]
Neighbor 3: [0. 2. 1. 5. 1. 0. 2. 3. 0. 0. 2.]
Neighbor 10: [0. 1. 0. 3. 0. 0. 5. 1. 5. 0. 5.]
Neighbor 13: [0. 3. 2. 5. 0. 0. 5. 4. 0. 2. 3.]


In [36]:
def find_aggregate_ratings_item(matrix, item_id, k):
    # Find the item's column
    item_index = items.index(item_id)
    item_column = matrix[:, item_index]

    # Find the k nearest neighbors
    neighbors = find_k_nearest_neighbors_item(matrix, item_id, k)

    # Find the unrated users
    unrated_users = np.where(item_column == 0)[0]
    print("Unrated users: ", unrated_users)

    # Calculate the aggregate ratings
    aggregate_ratings = []

    for user in unrated_users:
        ratings = []
        for neighbor, _ in neighbors:
            if matrix[user, neighbor] != 0:
                ratings.append(matrix[user, neighbor])
        if ratings:
            aggregate_rating = round(np.mean(ratings), 2)
            aggregate_ratings.append((user, aggregate_rating))
    
    return aggregate_ratings

In [37]:
print(find_aggregate_ratings_item(matrix, 'Item1', 3))

Similarities:  [(1, 4.0), (2, 4.0), (3, 2.27), (4, 5.73), (5, 6.64), (6, 5.73), (7, 3.82), (8, 4.18), (9, 7.55), (10, 2.27), (11, 5.27), (12, 8.18), (13, 2.45), (14, 4.91)]
Unrated users:  [2 4 5]
[(2, 1.5), (4, 1.0)]
